### Устанавливаем библиотеку для NLP русского языка

In [1]:
pip install yargy

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


### Загружаем данные из .csv файла в pandas dataframe(csv файл хранится в папке data).

In [2]:
import pandas as pd

In [3]:
filename = "data/test_data.csv"

In [4]:
df = pd.read_csv(filename, sep=",")

In [5]:
df['greeting'] = [0] * len(df)
df['goodbye'] = [0] * len(df)
df['self_intro'] = [0] * len(df)

### Делим датасет и создаём отдельные датасеты для каждого диалога

In [6]:
dialogues = []

In [7]:
for dialogue_id in df['dlg_id'].unique(): 
    dialogues.append(df[df['dlg_id']==dialogue_id].drop(['dlg_id'], axis=1, inplace=False).reset_index())

## Подключаем библиотеки

In [8]:
from yargy import Parser, rule, or_
from yargy.predicates import gram, dictionary
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact
from IPython.display import display

Извлекаем только слова manager, так как для задач тестового задания слова client не нужны

## a. Извлекать реплики с приветствием – где менеджер поздоровался.

In [9]:
GREETING = morph_pipeline([
    'привет',
    'добрый день',
    'доброе утро',
    'добрый вечер',
    'здравствуйте',
    'здравствуй'
])

parser_hi = Parser(GREETING)

In [10]:
for df_dial in dialogues:
    for index,text in df_dial[df_dial['role']=='manager']['text'].iteritems():
        for match in parser_hi.findall(text):
            df_dial.loc[index,'greeting'] = 1
            print('В реплике :"' + str(df_dial.loc[index,'text'])+ '" обнаружено приветствие:')
            display([_.value for _ in match.tokens])

В реплике :"Алло здравствуйте" обнаружено приветствие:


['здравствуйте']

В реплике :"Алло здравствуйте" обнаружено приветствие:


['здравствуйте']

В реплике :"Алло здравствуйте" обнаружено приветствие:


['здравствуйте']

В реплике :"Алло дмитрий добрый день" обнаружено приветствие:


['добрый', 'день']

## b. Извлекать реплики, где менеджер представил себя

In [11]:
Person = fact(
    'Person',
    ['selfintro', 'name']
)
Name = fact(
    'Name',['first'])

SELF_INTRO = morph_pipeline([
    'меня зовут',
    'я',
    'это'
])

NAME = rule(
    gram('Name').interpretation(
        Name.first.inflected()
    )
)

PERSON = rule(
    SELF_INTRO.interpretation(
        Person.selfintro
    ),
    NAME.interpretation(
        Person.name
    )
).interpretation(
    Person
)


parser_self_intro = Parser(PERSON)

In [12]:
for df_dial in dialogues:
    for index,text in df_dial[df_dial['role']=='manager']['text'].iteritems():
        for match in parser_self_intro.findall(text):
            df_dial.loc[index,'self_intro'] = 1
            print('В реплике :"' + str(df_dial.loc[index,'text'])+ '" менеджер представил себя:')
            display(match.fact)

В реплике :"Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается" менеджер представил себя:


Person(
    selfintro='Меня зовут',
    name='ангелина'
)

В реплике :"Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается" менеджер представил себя:


Person(
    selfintro='Меня зовут',
    name='ангелина'
)

В реплике :"Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там" менеджер представил себя:


Person(
    selfintro='Меня зовут',
    name='ангелина'
)

В реплике :"Добрый меня максим зовут компания китобизнес удобно говорить" менеджер представил себя:


Person(
    selfintro='меня',
    name='максим'
)

В реплике :"Да это анастасия" менеджер представил себя:


Person(
    selfintro='это',
    name='анастасия'
)

## c. Извлекать имя менеджера.

In [13]:
j = 0
for df_dial in dialogues:
    for text in df_dial[df_dial['role']=='manager']['text']:
        for match in parser_self_intro.findall(text):
            print('Менеджер диалога ' + str(j) + ':')
            display(match.fact.name)
    j+=1

Менеджер диалога 0:


'ангелина'

Менеджер диалога 1:


'ангелина'

Менеджер диалога 2:


'ангелина'

Менеджер диалога 3:


'максим'

Менеджер диалога 5:


'анастасия'

## d. Извлекать название компании

In [14]:
from yargy import or_
COMPANY = rule(
    dictionary({
        'компания'
    }),
    gram('NOUN'),
    or_(
        gram('NOUN'),
        gram('ADJF')
    ).optional().repeatable(),
)

parser_company = Parser(COMPANY)

In [15]:
j = 0
for df_dial in dialogues:
    for text in df_dial[df_dial['role']=='manager']['text']:
        for match in parser_company.findall(text):
            print('Компания из диалога ' + str(j) + ':')
            print([_.value for _ in match.tokens])
    j+=1

Компания из диалога 0:
['компания', 'диджитал', 'бизнес']
Компания из диалога 1:
['компания', 'диджитал', 'бизнес']
Компания из диалога 2:
['компания', 'диджитал', 'бизнес']
Компания из диалога 3:
['компания', 'китобизнес']


## e. Извлекать реплики, где менеджер попрощался.

In [16]:
GOODBYE = morph_pipeline([
    'до свидания',
    'всего хорошего'
])

parser_bye = Parser(GOODBYE)

In [17]:
for df_dial in dialogues:
    for index,text in df_dial[df_dial['role']=='manager']['text'].iteritems():
        for match in parser_bye.findall(text):
            df_dial.loc[index,'goodbye'] = 1
            print('В реплике :"' + str(df_dial.loc[index,'text'])+ '" менеджер попрощался:')
            display([_.value for _ in match.tokens])

В реплике :"Всего хорошего до свидания" менеджер попрощался:


['Всего', 'хорошего']

В реплике :"Всего хорошего до свидания" менеджер попрощался:


['до', 'свидания']

В реплике :"Угу да вижу я эту почту хорошо тогда исправлю на эту будем ждать ответа всего хорошего" менеджер попрощался:


...

['до', 'свидания']

В реплике :"Все хорошо я вас поняла тогда вы хотите сейчас полностью обновиться и уже начать правильно работать" менеджер попрощался:


['Все', 'хорошо']

В реплике :"Ну до свидания хорошего вечера" менеджер попрощался:


['до', 'свидания']

## f. Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом

In [18]:
j = 0
for df_dial in dialogues:
    if 1 in df_dial['greeting'].values and 1 in df_dial['goodbye'].values:
        #print(df_dial['greeting'])
        print('Менеджер диалога ' + str(j) + ' поздоровался и попрощался')
    j+=1

Менеджер диалога 0 поздоровался и попрощался
Менеджер диалога 1 поздоровался и попрощался
Менеджер диалога 2 поздоровался и попрощался
Менеджер диалога 3 поздоровался и попрощался
